In [3]:
import shapefile
from shapely.geometry import shape, Point
import numpy as np
import pandas as pd

base = "../../data/raw/"
filename = "2016_LGA_SHAPE/LGA_2016_AUST"

sf = shapefile.Reader(base+filename)

nswshapes = list(map(lambda x: shape(x.__geo_interface__), sf.shapes()[:130]))

In [110]:
import json
lga_json = list(map(lambda x: x.__geo_interface__, sf.shapes()[:130]))

geojson = {'type': 'FeatureCollection',
            'features': [{'type':'Feature', 'geometry':x, 'properties': {'name':nswdf.iloc[i]['clean_name']}} for i, x in enumerate(lga_json)]
          }


with open('geojson.json','w') as f:
    json.dump(geojson, f)
#geojson[:5]

In [4]:
import pandas as pd
df = pd.DataFrame(sf.records())
nswdf = df[df[4] =="New South Wales"]
# drop "No usual address" - not a useful LGA
nswdf = nswdf[nswdf[0] != "LGA19499"]
nswdf.columns = ["code", "codenum", "name", "unknown", "state", "unknown2"]
nswdf["clean_name"] = nswdf["name"].str.extract('([^\\(]*)', expand=False).str.strip()
nswdf



,code,codenum,name,unknown,state,unknown2,clean_name
0,LGA10050,10050,Albury (C),1,New South Wales,305.9459,Albury
1,LGA10130,10130,Armidale Regional (A),1,New South Wales,8620.6990,Armidale Regional
2,LGA10250,10250,Ballina (A),1,New South Wales,484.9389,Ballina
3,LGA10300,10300,Balranald (A),1,New South Wales,21690.6753,Balranald
4,LGA10470,10470,Bathurst Regional (A),1,New South Wales,3817.8646,Bathurst Regional
5,LGA10550,10550,Bega Valley (A),1,New South Wales,6278.8811,Bega Valley
6,LGA10600,10600,Bellingen (A),1,New South Wales,1600.4337,Bellingen
7,LGA10650,10650,Berrigan (A),1,New South Wales,2065.7759,Berrigan
8,LGA10750,10750,Blacktown (C),1,New South Wales,240.0487,Blacktown
9,LGA10800,10800,Bland (A),1,New South Wales,8557.6535,Bland


In [119]:
target = pd.read_csv(base+"cleaned_target.csv")

set(nswdf["clean_name"]).difference(set(target["LGA"]))
#set(target["LGA"]).difference(set(nswdf["clean_name"]))

{'Botany Bay',
 'Gundagai',
 'Rockdale',
 'Unincorporated NSW',
 'Western Plains Regional'}

In [46]:
import os

tweetdf =pd.read_csv(os.path.join(base, "tweet.csv"),header=0)



In [47]:
loctweetdf = tweetdf.loc[tweetdf['lat'].notnull(),:]

In [48]:
loctweetdf

,id,time,created_at,from_user_name,text,filter_level,possibly_sensitive,withheld_copyright,withheld_scope,truncated,...,from_user_utcoffset,from_user_timezone,from_user_lang,from_user_tweetcount,from_user_followercount,from_user_friendcount,from_user_favourites_count,from_user_listed,from_user_withheld_scope,from_user_created_at
1,951300448031473664,1515642675,2018-01-11 03:51:15,Markfchris,Things to do on hot day in Sydney. @ Carriagew...,low,0.0,NaN,NaN,0,...,39600.0,Sydney,en,14953,695,1725,2615,62,NaN,2010-04-22 02:37:40
4,951300560812113920,1515642702,2018-01-11 03:51:42,ZKII_,BB And The Blips x The Red Rattler Theater @ T...,low,0.0,NaN,NaN,0,...,NaN,NaN,en,604,86,275,134,3,NaN,2012-10-02 09:55:23
10,951300678298886144,1515642730,2018-01-11 03:52:10,AlexandraAndoni,Client work takes us all over the map - today ...,low,0.0,NaN,NaN,0,...,28800.0,Hong Kong,en,995,210,464,489,29,NaN,2011-04-14 09:16:44
11,951300740743655424,1515642745,2018-01-11 03:52:25,JensensTennis,Awesome day at the Sydney International Wheelc...,low,0.0,NaN,NaN,0,...,-28800.0,Pacific Time (US & Canada),en,49,13,94,83,0,NaN,2017-11-02 22:03:22
17,951300915641901056,1515642787,2018-01-11 03:53:07,pavel6067,Drinking a Vilkmerges Sviesusis by Vilkmerges ...,low,0.0,NaN,NaN,0,...,NaN,NaN,en,18843,40,90,50,11,NaN,2011-07-02 14:17:21
18,951300995216297989,1515642806,2018-01-11 03:53:26,AndieSoetaman,enjoying the view on top of a cliff next to Bo...,low,0.0,NaN,NaN,0,...,39600.0,Sydney,en,8448,1283,251,399,9,NaN,2010-01-27 12:17:14
30,951301525636354048,1515642932,2018-01-11 03:55:32,Russy_Bear,🏄🏼‍♂️🏖#bondi #bondibeach #bondirescue #streeta...,low,0.0,NaN,NaN,0,...,NaN,NaN,en,36,9,32,24,0,NaN,2014-01-26 20:17:12
40,951301855107342336,1515643011,2018-01-11 03:56:51,ATOAWASCA,Estoy autorizado a setirme bonito? @ Coogee Be...,low,0.0,NaN,NaN,0,...,-18000.0,Quito,en,109878,2177,2052,4630,7,NaN,2010-01-30 02:48:16
46,951302438291722241,1515643150,2018-01-11 03:59:10,recycologyy,Farewell to the #elvisexpress leaving #central...,low,0.0,NaN,NaN,0,...,39600.0,Sydney,en,8875,300,75,14,52,NaN,2011-02-14 03:23:56
50,951302658366861318,1515643202,2018-01-11 04:00:02,susan_paget,A little update on my quest to take a break fr...,low,0.0,NaN,NaN,0,...,36000.0,Solomon Is.,en,3761,356,255,502,26,NaN,2012-09-21 07:25:31


In [120]:
def whichlga(tweetpoints, lgashapes):
    #loop over 130 NSW LGAs and return a name or "None"
    #NB you get names from Census data which is 2016 - pre amalgamations
    #test by whether in bounding box. If in two, go to shapes. If still in two, one with smaller bounding box
    output = []
    nums = []
    amalgamations = {'Botany Bay': 'Bayside',
                     'Rockdale'  : 'Bayside',
                     'Gundagai'  : 'Cootamundra-Gundagai',
                     'Western Plains Regional' : 'Dubbo Regional',
                     'Unincorporated NSW' : 'Unincorporated Far West'
                    }
    
    for point in tweetpoints:
        found = []
        distances = []
        for i, nswshape in enumerate(nswshapes):
            if point.within(nswshape):
                #import pdb; pdb.set_trace()
                found.append(i)
                break
            else:
                distances.append(point.distance(nswshape))
                
        #print(len(found))
        nums.append(len(found))
        if len(found) == 1:
            clean_name = nswdf.iloc[found[0]].clean_name
        else:
            #out on the water
            clean_name = nswdf.iloc[np.argmin(np.asarray(distances))].clean_name
            
        if clean_name in amalgamations:
            clean_name = amalgamations[clean_name]
        output.append(clean_name)
            
    return pd.Series(output), nums
            

In [50]:
tweetpoints = []

loctweetdf.apply(lambda x: tweetpoints.append(shape(Point(x['lng'], x['lat']))), axis=1)


1         None
4         None
10        None
11        None
17        None
18        None
30        None
40        None
46        None
50        None
54        None
57        None
58        None
72        None
73        None
74        None
75        None
76        None
78        None
84        None
90        None
98        None
105       None
112       None
154       None
162       None
164       None
165       None
167       None
185       None
          ... 
522271    None
522282    None
522293    None
522294    None
522296    None
522310    None
522314    None
522330    None
522337    None
522345    None
522356    None
522375    None
522383    None
522384    None
522385    None
522386    None
522387    None
522388    None
522389    None
522391    None
522399    None
522409    None
522411    None
522416    None
522417    None
522420    None
522422    None
522434    None
522438    None
522439    None
Length: 75082, dtype: object

In [121]:

output, found = whichlga(tweetpoints, nswshapes)
found = np.asarray(found)
np.histogram(found)

(array([15725,     0,     0,     0,     0,     0,     0,     0,     0,
        59357]),
 array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]))

In [114]:
#tweetdf[pd.isnull(output)]
tweetdf.loc[tweetdf['lat'].notnull(),'lga'] = np.asarray(output)


In [95]:
tweetdf.to_csv("tweets_w_lga.csv")

In [115]:
nextaggdf = tweetdf[['lat','lng','text','lga']].loc[pd.isnull(tweetdf['lga']) & ~pd.isnull(tweetdf['lat'])].groupby(['lat','lng']).agg('count').reset_index()
#output


In [117]:
tweetdf[['lat','lng','text','lga']].loc[~pd.isnull(tweetdf['lat'])]

,lat,lng,text,lga
1,-33.893658,151.192032,Things to do on hot day in Sydney. @ Carriagew...,Sydney
4,-33.909939,151.163666,BB And The Blips x The Red Rattler Theater @ T...,Inner West
10,-33.777199,151.124115,Client work takes us all over the map - today ...,Ryde
11,-33.718742,150.922531,Awesome day at the Sydney International Wheelc...,Blacktown
17,-33.807701,151.089005,Drinking a Vilkmerges Sviesusis by Vilkmerges ...,Ryde
18,-33.888908,151.260635,enjoying the view on top of a cliff next to Bo...,Waverley
30,-33.888882,151.277588,🏄🏼‍♂️🏖#bondi #bondibeach #bondirescue #streeta...,Waverley
40,-33.920692,151.257263,Estoy autorizado a setirme bonito? @ Coogee Be...,Randwick
46,-33.882500,151.206665,Farewell to the #elvisexpress leaving #central...,Sydney
50,-33.797215,151.287430,A little update on my quest to take a break fr...,Northern Beaches


In [ ]:
import dash
import dash_core_components as dcc
from plotly.graph_objs import *

app = dash.Dash()

colorscale = [
    'rgb(193, 193, 193)',
    'rgb(239,239,239)',
    'rgb(195, 196, 222)',
    'rgb(144,148,194)',
    'rgb(101,104,168)',
    'rgb(65, 53, 132)'
]

lga_layers = list(map(lambda x: { 
                'id' : x['properties']['name'],
                'sourcetype' : "geojson",
                'source': x,
                'type'   : 'line',
                'layout': {
                    'line-join': 'round',
                    'line-cap' : 'round'
                },
                'line-color': 'lightblue',
                'line-width': 4
                }, geojson['features']))

data = [dict(
        type = 'scattermapbox',
        lon = aggdf['lng'],
        lat = aggdf['lat'],
        mode = 'markers',
        marker = dict(
            size = 8,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'circle',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = colorscale,
            cmin = 0,
            color = aggdf['text'],
            cmax = aggdf['text'].max(),
            colorbar=dict(
                title="Tweets not in an LGA"
            )
        ))]

app.layout = dcc.Graph(
    id = "mapbox",
    
    figure={
        "data": data,
        "layout": dict(
            hovermode = "closest",
            margin = dict(l = 0, r = 0, t = 0, b = 0),
            mapbox = dict(
                layers = lga_layers,
                accesstoken = "pk.eyJ1IjoiZGVzdWx0aXIiLCJhIjoiY2pmZDN4M2FlMTEzYjJ3bWlyc2VhaDkxaiJ9.VvezwDLTQlh2GVAEsauwlg",
                bearing = 0,
                center = dict(lat=-32.162833, lon=147.031917),
                style = "light",
                pitch = 0,
                zoom = 6.0,
            )
        )
    },
    style = {"height": 750}
)

app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Mar/2018 14:03:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2018 14:03:29] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2018 14:03:29] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2018 14:03:29] "GET /favicon.ico HTTP/1.1" 200 -


In [131]:
lga_layers[0]

{'id': 'Albury',
 'layout': {'line-cap': 'round', 'line-join': 'round'},
 'paint': {'line-color': '#888', 'line-width': 8},
 'source': {'data': '{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[146.81766133100007, -36.06740560999998], [146.82137801600004, -36.04958200699997], [146.82187354000007, -36.04976034699996], [146.82530702300005, -36.04946099899996], [146.82645502000003, -36.04724099899994], [146.825958, -36.04357999699994], [146.82735000000002, -36.03687200799993], [146.82775799700005, -36.036926008999956], [146.8312110070001, -36.03329399699993], [146.83045501900006, -36.03221999799996], [146.83159998700012, -36.02901400299993], [146.8320260150001, -36.02885399699994], [146.83222300600005, -36.02868399999994], [146.83316339200007, -36.02378131499995], [146.83341943900007, -36.02363013299998], [146.8342913150001, -36.01922859499996], [146.83407012500004, -36.019203767999954], [146.83474840400004, -36.015632582999956], [146.83899974200006, -36.0161363569999